<a href="https://colab.research.google.com/github/jmhuer/shift_invariant_dictionary_learning/blob/main/dictionarylearning_midipiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up enviroment


###Setup Environment and Dependencies. Check GPU.

In [3]:
#@title Check if GPU (driver) is avaiiable (you do not want to run this on CPU, trust me)
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Sun Jul 11 15:28:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |         

In [4]:
#@title Clone/Install all dependencies
!git clone https://github.com/asigalov61/midi-neural-processor
!git clone https://github.com/asigalov61/MusicTransformer-Pytorch
!pip install tqdm
!pip install progress
!pip install pretty-midi
!pip install pypianoroll
!pip install matplotlib
!pip install librosa
!pip install scipy
!pip install pillow
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!pip install mir_eval
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 /content/font.sf2

Cloning into 'midi-neural-processor'...
remote: Enumerating objects: 26, done.
remote: Total 26 (delta 0), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (26/26), done.
fatal: destination path 'MusicTransformer-Pytorch' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (1.1.9-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [5]:
#@title Import all needed modules
import numpy as np
import pickle
import os
import sys
import math
import random
# For plotting
import pypianoroll
from pypianoroll import Multitrack, Track
import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('SVG')
#%matplotlib inline
#matplotlib.get_backend()
import mir_eval.display
import librosa
import librosa.display
# For rendering output audio
import pretty_midi
from midi2audio import FluidSynth
from google.colab import output
from IPython.display import display, Javascript, HTML, Audio

#Option 1: MAESTRO DataSet

In [6]:
#@title Download Google Magenta MAESTRO v.2.0.0 Piano MIDI Dataset (~1300 MIDIs)
%cd /content/MusicTransformer-Pytorch/dataset/
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip maestro-v2.0.0-midi.zip
%cd /content/MusicTransformer-Pytorch/

/content/MusicTransformer-Pytorch/dataset
--2021-07-11 15:28:27--  https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59243107 (56M) [application/zip]
Saving to: ‘maestro-v2.0.0-midi.zip.2’

maestro-v2.0.0-midi 100%[===================>]  56.50M   230MB/s    in 0.2s    

2021-07-11 15:28:27 (230 MB/s) - ‘maestro-v2.0.0-midi.zip.2’ saved [59243107/59243107]

Archive:  maestro-v2.0.0-midi.zip
replace maestro-v2.0.0/maestro-v2.0.0.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: /content/MusicTransformer-Pytorch


In [7]:
#@title Prepare directory sctructure and MIDI processor
%cd /content/
!mv midi-neural-processor midi_processor
%cd /content/MusicTransformer-Pytorch/

/content
mv: cannot move 'midi-neural-processor' to 'midi_processor/midi-neural-processor': Directory not empty
/content/MusicTransformer-Pytorch


In [8]:
#@title Process MAESTRO MIDI DataSet
!python3 preprocess_midi.py '/content/MusicTransformer-Pytorch/dataset/maestro-v2.0.0'

Preprocessing midi files and saving to ./dataset/e_piano
Found 1282 pieces
Preprocessing...
50 / 1282
100 / 1282
Num Train: 93
Num Val: 18
Num Test: 20
Done!



In [11]:
from processor import encode_midi, decode_midi

                  
from dataset.e_piano import create_epiano_datasets, compute_epiano_accuracy


train_dataset, val_dataset, test_dataset = create_epiano_datasets("/content/MusicTransformer-Pytorch/dataset/e_piano", 2048)

example1 = list(train_dataset)[1].numpy()
print(example1)
print("torch size ", train_dataset.size())

# print(len(list(train_dataset)[9][0]))
# tmp = []
name = "test111"
# for point in train_dataset:
#     # isthis = decode_midi(point[0].numpy(), name + ".mid")
#     isthis = decode_midi(point[0].numpy())
#     tmp.append(isthis.estimate_tempo())


# print("tempo:" , tmp )

decode_midi(example1[0:2048], name + ".mid")
FluidSynth("/content/font.sf2").midi_to_audio(name + ".mid", name + ".wav")
Audio(name + ".wav")


[355 256 365 ... 256 191 191]


AttributeError: ignored

In [15]:
from processor import encode_midi, decode_midi
import torch
                  
from dataset.e_piano import create_epiano_datasets, compute_epiano_accuracy


train_dataset, val_dataset, test_dataset = create_epiano_datasets("/content/MusicTransformer-Pytorch/dataset/e_piano", 2048)

for i in range(len(train_dataset)):
    print("train_dataset size", train_dataset[i].size())



train_dataset size torch.Size([49718])
train_dataset size torch.Size([16670])
train_dataset size torch.Size([17427])
train_dataset size torch.Size([16605])
train_dataset size torch.Size([39613])
train_dataset size torch.Size([20844])
train_dataset size torch.Size([64828])
train_dataset size torch.Size([66370])
train_dataset size torch.Size([11528])
train_dataset size torch.Size([28377])
train_dataset size torch.Size([48131])
train_dataset size torch.Size([46987])
train_dataset size torch.Size([21366])
train_dataset size torch.Size([10020])
train_dataset size torch.Size([30414])
train_dataset size torch.Size([20936])
train_dataset size torch.Size([11575])
train_dataset size torch.Size([13870])
train_dataset size torch.Size([13295])
train_dataset size torch.Size([29672])
train_dataset size torch.Size([51452])
train_dataset size torch.Size([40244])
train_dataset size torch.Size([25488])
train_dataset size torch.Size([13570])
train_dataset size torch.Size([16406])
train_dataset size torch.

# Model definitions

In [17]:
#@title Pytorch for DL

import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import torch
from torch.nn.utils import weight_norm
import numpy as np

def get_model_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

In [18]:
#@title KWTA


class SparsifyBase(nn.Module):
    def __init__(self, sparse_ratio=0.5):
        super(SparsifyBase, self).__init__()
        self.sr = sparse_ratio
        self.preact = None
        self.act = None
    def get_activation(self):
        def hook(model, input, output):
            self.preact = input[0].cpu().detach().clone()
            self.act = output.cpu().detach().clone()
        return hook
    def record_activation(self):
        self.register_forward_hook(self.get_activation())


class Sparsify1D_kactive(SparsifyBase):
    def __init__(self, k=1):
        super(Sparsify1D_kactive, self).__init__()
        self.k = k
    def forward(self, x):
        m = torch.zeros(x.shape).to(device)
        for i in range(self.k):
            indeces = x.topk(self.k, dim=1)[1][:, i]
            m += torch.mul(torch.zeros(x.shape).to(device).scatter(1, indeces.unsqueeze(1), 1), x)
            # print("\n hi", m )
        return m.double()

In [19]:
#@title TCN 

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 1)
        self.conv2.weight.data.normal_(0, 1)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 1)

    def forward(self, x):
        # print("block ", x.size())
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)


        # print("last layer conv", self.network[-1].conv2.weight.data[:,0,:].size())
        # print("last layer conv", self.network[-1].conv2.weight.data[:,0,:])

    def forward(self, x):
        return self.network(x)


In [44]:
#@title TCN - Autoeconder 

class TCNAutoencoder(nn.Module):
    def __init__(self, kernel_size, dropout, wta_k):
        super(TCNAutoencoder, self).__init__()
        self.wta = Sparsify1D_kactive(k = wta_k)
        # self.feature = TemporalConvNet(input_size, num_channels, kernel_size, dropout=dropout).double()
        self.encoder = torch.nn.Conv1d(in_channels=1, out_channels=1000, kernel_size=4, padding=0, bias=False, stride=4)
        self.decoder = torch.nn.ConvTranspose1d(in_channels=1000, out_channels=1, kernel_size=4, padding=0, bias=False, stride=4)
        # self.encoder.weight.data.normal(1)
        self.relu1 = nn.ReLU()
        self.code = None
        # torch.nn.init.xavier_uniform(self.encoder.weight)
        # torch.nn.init.xavier_uniform(self.decoder.weight)
    def get_kernels(self):
        return self.decoder.weight.data[:,0,:]
    def feature_map(self, x):
        code = self.code
        return code
    def forward(self, x):
        # x needs to have dimension (N, C, L) in order to be passed into CNN
        # output = self.feature(x)
        self.code = self.wta(self.encoder(x))
        output = self.decoder(self.code )
        return output


# Model training 

In [ ]:
#@title GO

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device: ", device)

model = TCNAutoencoder(kernel_size=4, 
                       dropout=0.2, 
                       wta_k = 30).to(device).double()
print("TCNAutoencoder trainable parameters: ", get_model_parameters(model))



loss_fn = torch.nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=.005,  betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True) ##this has weight decay just like you implemented
epochs = 100
history = {"loss": []}
print("test " ,max(1, model.wta.k - 1) )
for i in range(epochs):
    #decaying WTA
    if i % 10 == 0 and i != 0:
        model.wta.k = max(1, model.wta.k - 1)
        print("model.wta.k: ", model.wta.k)
    for train_data in train_dataset:
      lenby4 = len(train_data) // 4
      train_data = train_data[None, None, 0:lenby4*4].to(device).double()
      optimizer.zero_grad()
      output = model(train_data)

      loss = loss_fn(output, train_data)
      loss.backward()
      # torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
      optimizer.step()
      history["loss"].append(float(loss))
      if i % 1 == 0:
          print("Epoch : {} \t Loss : {} \t Code_Sparsity: {} ".format(i, round(float(loss),7), torch.count_nonzero(model.code)))
          





Streaming output truncated to the last 5000 lines.
Epoch : 5 	 Loss : 234.5371489 	 Code_Sparsity: 104010 
Epoch : 5 	 Loss : 217.1630254 	 Code_Sparsity: 99690 
Epoch : 5 	 Loss : 289.7655184 	 Code_Sparsity: 222540 
Epoch : 5 	 Loss : 257.0264456 	 Code_Sparsity: 385890 
Epoch : 5 	 Loss : 276.3082268 	 Code_Sparsity: 301830 
Epoch : 5 	 Loss : 297.6038932 	 Code_Sparsity: 191160 
Epoch : 5 	 Loss : 223.8754022 	 Code_Sparsity: 101760 
Epoch : 5 	 Loss : 239.621844 	 Code_Sparsity: 123030 
Epoch : 5 	 Loss : 262.5276899 	 Code_Sparsity: 354690 
Epoch : 5 	 Loss : 285.0046388 	 Code_Sparsity: 367260 
Epoch : 5 	 Loss : 252.967466 	 Code_Sparsity: 292830 
Epoch : 5 	 Loss : 234.2137437 	 Code_Sparsity: 126300 
Epoch : 5 	 Loss : 230.054504 	 Code_Sparsity: 125220 
Epoch : 5 	 Loss : 256.5284138 	 Code_Sparsity: 357000 
Epoch : 5 	 Loss : 226.3889037 	 Code_Sparsity: 224940 
Epoch : 5 	 Loss : 242.6155899 	 Code_Sparsity: 163110 
Epoch : 5 	 Loss : 244.1945018 	 Code_Sparsity: 342150 
E

In [ ]:
#@title Test Recustruction 

def get_code(model,input):
    model(input)
    return model.code

def play_example(input):
    decode_midi(input[0:2048], name + ".mid")
    FluidSynth("/content/font.sf2").midi_to_audio(name + ".mid", name + ".wav")
    Audio(name + ".wav")


#make it a keep top n 
def exchange_max_rows(A,B):
    maxrowA = torch.argmax(A.sum(1))
    maxrowB = torch.argmax(B.sum(1))
    rowA = A[maxrowA:maxrowA+1,].clone()
    rowB = B[maxrowB:maxrowB+1,].clone()
    # print("rowA ",rowA )
    # print("rowB ",rowB )
    A[maxrowB:maxrowB+1,] =  rowB
    B[maxrowA:maxrowA+1,] =  rowA
    return A,B

#make it a keep top n 
def keep_topk(A,k):
    mask = torch.zeros(A.shape).to(device)
    v, i  = torch.topk(A.sum(1), k)
    print("\n index is", i)
    mask[i, ] = True
    return mask * A




In [ ]:
    
index_example = 1432


#get raw_input
raw_input = train_dataset[index_example] 


#example 1
print("orginal 1")
input1 = torch.cat([raw_input], axis=-1)
play_example(input1)



